In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
from datetime import datetime,timedelta
import re
import math

In [2]:
#'''文件路径定义'''
ACTION_FILE = './datafile/JData_Action.csv'
COMMENT_FILE = './datafile/JData_Comment.csv'
PRODUCT_FILE = './datafile/JData_Product.csv'
USER_FILE = './datafile/JData_User.csv'

In [3]:
#4.15或者4.14号登陆了并且这两个多月登陆天数大于1天
def last_day_load_and_outnumber_two_days():
    if os.path.exists('./cache/buy_user_browse_count'):
        with open('./cache/buy_user_browse_count','rb') as f:
            df = pickle.load(f)
    else:
        dump_path = './cache/Action_2016-02-01_2016-04-16_2'
        with open(dump_path,'rb') as f:
            df = pickle.load(f)
        last_two_days = df[(df.load_days<=2)&(df.loc[:,'2016-04-14']==1)|(df.load_days<=2)&(df.loc[:,'2016-04-15']==1)]
        buy_user_id = last_two_days['user_id']
        buy_user_id = pd.DataFrame(buy_user_id)
        path = './cache/Action_2016-02-01_2016-04-16_with_sku_id-time-user_id'
        with open(path,'rb') as f:
            buy_sku_id = pickle.load(f)
        buy_sku_id = buy_sku_id.drop_duplicates()
        buy_user_sku_id = pd.merge(buy_user_id,buy_sku_id,on='user_id',how='left').drop_duplicates()
        buy_user_sku_id['time'] = buy_user_sku_id.time.apply(lambda t :t[:10])
        buy_user_sku_id = buy_user_sku_id[buy_user_sku_id.time>='2016-04-14']
        df = buy_user_sku_id.groupby(['user_id','sku_id']).sku_id.count()
        df = pd.DataFrame(df)
        with open('./cache/buy_user_browse_count','wb') as f:
            pickle.dump(df,f)
    return df

In [4]:
#筛选出最后两天登陆并且已经购买了商品的用户
def not_buy_user():
    path = './cache/Action_2016-02-01_2016-04-16_with_sku_id-time-type-user_id'
    with open(path,'rb') as f:
        df = pickle.load(f)
    buy_user_id = pd.read_csv('aa.csv')['user_id']
    buy_user_id = pd.DataFrame(buy_user_id)
    df = pd.merge(buy_user_id,df,on='user_id',how='left')
    df['time'] = df.time.apply(lambda t:t[:10])
    last_two_days = df[df['time']>='2016-04-14']
    last_two_days = last_two_days[last_two_days.type==4][['user_id','sku_id']]
    trust_buy = pd.merge(buy_user_id,last_two_days,on='user_id',how='left')
    trust_buy = trust_buy.fillna(value=0)
    user_id = trust_buy[trust_buy.sku_id==0]['user_id']
    user_id = pd.DataFrame(user_id) 
    with open('./cache/buy_tommrow','wb') as f:
        pickle(user_id,f)
    return user_id

In [3]:
dump_path = './cache/Action_2016-02-01_2016-04-16_2'
if os.path.exists(dump_path):
    with open(dump_path,'rb') as f:
        df = pickle.load(f)
df

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days
0,200001.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,1,22
1,200002.0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,19
2,200003.0,0,1,1,0,0,0,0,0,0,...,1,0,0,1,0,0,1,1,0,13
3,200004.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,4
4,200005.0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,9
5,200006.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,7
6,200007.0,0,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,8
7,200008.0,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,12
8,200009.0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,1,8
9,200010.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


# 登陆天数大于60天的，基本上不买

In [9]:
#查看登陆天数小于60天大于45天的用户购买轨迹
load_45_days = df[(df.load_days>45)&(df.load_days<=60)]
load_45_days

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days
117,200118.0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,52
241,200242.0,1,1,1,1,0,1,0,0,0,...,1,0,1,1,1,1,1,1,0,51
247,200248.0,1,0,1,1,1,1,0,0,0,...,1,1,1,0,0,1,1,1,1,52
259,200260.0,0,0,1,1,1,1,0,1,0,...,1,1,1,1,0,0,1,1,1,55
335,200336.0,0,1,1,1,0,0,0,0,0,...,1,1,0,1,1,1,1,1,0,46
375,200376.0,1,1,1,1,1,0,1,1,1,...,1,0,1,1,1,1,1,0,1,54
412,200413.0,1,1,1,1,1,1,0,0,0,...,1,1,1,1,1,0,1,0,1,51
470,200471.0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,1,0,1,1,47
474,200475.0,0,0,0,1,1,1,1,1,0,...,0,1,0,0,1,0,0,0,0,46
485,200486.0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,1,0,1,1,1,46


# 登陆次数少的，看看他们的购买情况

In [4]:
# load_2_10_days = df[(df.load_days>2)&(df.load_days<=10)]
# load_45_days = load_2_10_days
load_45_days = df

In [5]:
user_level = pd.read_csv(USER_FILE,encoding='gbk')
load_45_days = pd.merge(load_45_days,user_level,how='left',on='user_id')[['user_id','load_days','age','sex','user_lv_cd','user_reg_tm','2016-04-12','2016-04-13','2016-04-14','2016-04-15']]
load_45_days

,user_id,load_days,age,sex,user_lv_cd,user_reg_tm,2016-04-12,2016-04-13,2016-04-14,2016-04-15
0,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1
1,200002.0,19,-1,0.0,1,2016-01-26,0,1,1,0
2,200003.0,13,36-45岁,1.0,4,2016-01-26,0,1,1,0
3,200004.0,4,-1,2.0,1,2016-01-26,1,0,1,0
4,200005.0,9,16-25岁,0.0,4,2016-01-26,0,0,0,0
5,200006.0,7,36-45岁,2.0,2,2013-04-10,0,0,0,1
6,200007.0,8,36-45岁,2.0,3,2016-01-26,0,0,0,0
7,200008.0,12,-1,2.0,3,2016-01-26,0,0,0,0
8,200009.0,8,36-45岁,2.0,2,2016-01-26,1,0,1,1
9,200010.0,4,36-45岁,2.0,3,2016-01-26,0,0,0,0


In [6]:
with open('./cache/Action_2016-02-01_2016-04-16_with_time-type-user_id','rb') as f:
    user_action = pickle.load(f)
user_action = pd.merge(load_45_days,user_action,on='user_id',how='left')
user_action

,user_id,load_days,age,sex,user_lv_cd,user_reg_tm,2016-04-12,2016-04-13,2016-04-14,2016-04-15,time,type
0,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1,2016-03-13 12:44:11,6
1,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1,2016-03-13 12:44:11,1
2,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1,2016-03-13 12:44:11,1
3,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1,2016-03-13 12:44:11,6
4,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1,2016-03-13 12:44:11,6
5,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1,2016-03-13 12:44:11,6
6,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1,2016-03-13 12:44:41,6
7,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1,2016-03-13 12:44:41,1
8,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1,2016-03-13 12:44:41,6
9,200001.0,22,56岁以上,2.0,5,2016-01-26,0,1,0,1,2016-03-13 12:44:41,1


In [96]:
b=user_action[user_action.type==4][['user_id','time','user_lv_cd','age','sex','user_reg_tm','load_days','type']]
b.sort_values(by='load_days',ascending=True)

,user_id,time,user_lv_cd,age,sex,user_reg_tm,load_days,type
7510351,279055.0,2016-03-25 16:41:00,4,26-35岁,2.0,2015-05-29,3,4
1431871,212379.0,2016-03-22 15:37:00,5,26-35岁,0.0,2010-10-12,3,4
1431870,212379.0,2016-03-22 15:37:00,5,26-35岁,0.0,2010-10-12,3,4
1431869,212379.0,2016-03-22 15:37:00,5,26-35岁,0.0,2010-10-12,3,4
1429177,212347.0,2016-03-02 16:42:00,5,26-35岁,0.0,2008-12-10,3,4
1424804,212312.0,2016-02-25 15:17:00,5,36-45岁,2.0,2013-07-22,3,4
6231221,265582.0,2016-02-22 18:22:00,3,26-35岁,2.0,2015-02-02,3,4
3894653,240248.0,2016-03-15 10:28:00,4,36-45岁,0.0,2011-11-24,3,4
6226247,265534.0,2016-04-07 18:33:00,4,-1,0.0,2015-02-02,3,4
6221233,265461.0,2016-03-03 12:41:00,3,36-45岁,2.0,2015-02-01,3,4


In [97]:
c = b[['user_id','type']]
c = c.groupby(['user_id'],as_index=False).sum()
c['type'] = c['type']/4
c.sort_values(by='type',ascending=False)

,user_id,type
7067,267775.0,11.0
374,203507.0,11.0
19,200258.0,10.0
7,200077.0,10.0
9815,296743.0,9.0
2488,222511.0,9.0
2334,221157.0,9.0
10385,302895.0,8.0
636,205196.0,8.0
1755,214974.0,8.0


In [32]:
len(c[c.type>9])

12

# 将C中买的次数大于2的用户取出来，查看这类用户在最后两天有没有加购物车

In [98]:
c_big_2 = c
c_big_2.columns = ['user_id','buy_numbers']
with open('./cache/Action_2016-02-01_2016-04-16_with_time-type-user_id','rb') as f:
    user_action = pickle.load(f)
c_big_2 = pd.merge(c_big_2,user_action,on='user_id',how='left')
#4-14和4-15号的信息
c_big_2 = c_big_2[c_big_2.time>'2016-04-14 00:00:00']
c_big_2 = c_big_2[(c_big_2.type!=4)]
c_big_2

,user_id,buy_numbers,time,type
734,200034.0,1.0,2016-04-14 08:12:03,1
735,200034.0,1.0,2016-04-14 08:12:03,6
736,200034.0,1.0,2016-04-14 08:12:03,1
737,200034.0,1.0,2016-04-14 08:12:03,6
738,200034.0,1.0,2016-04-14 08:12:03,6
739,200034.0,1.0,2016-04-14 08:12:59,6
740,200034.0,1.0,2016-04-14 08:12:59,6
741,200034.0,1.0,2016-04-14 08:12:59,1
742,200034.0,1.0,2016-04-14 08:12:59,6
743,200034.0,1.0,2016-04-14 08:12:59,6


In [84]:
item = user_action[user_action['user_id']==299521.0]
item = item[(item.time>'2016-04-13 00:00:00')]
len(item[item.type==4])

0

In [99]:
c_big_2 = c_big_2[c_big_2.type==2]
c_big_2

,user_id,buy_numbers,time,type
749,200034.0,1.0,2016-04-14 08:14:03,2
1273,200074.0,1.0,2016-04-14 18:58:37,2
11786,200512.0,1.0,2016-04-14 00:01:32,2
11802,200512.0,1.0,2016-04-14 00:12:14,2
11842,200512.0,1.0,2016-04-14 00:36:35,2
16986,200721.0,1.0,2016-04-14 00:19:00,2
17668,200728.0,1.0,2016-04-14 11:38:17,2
17683,200728.0,1.0,2016-04-14 11:59:35,2
23593,201092.0,1.0,2016-04-14 11:47:32,2
23600,201092.0,1.0,2016-04-14 11:48:19,2


In [100]:
type_c = pd.get_dummies(c_big_2['type'],prefix='type')
will_buy = c_big_2['user_id']
will_buy = pd.DataFrame(will_buy)
will_buy = pd.concat([will_buy,type_c],axis=1)
will_buy = will_buy.groupby('user_id',as_index=False).sum()
will_buy

,user_id,type_2
0,200034.0,1.0
1,200074.0,1.0
2,200512.0,3.0
3,200721.0,1.0
4,200728.0,2.0
5,201092.0,2.0
6,201126.0,4.0
7,201350.0,2.0
8,201529.0,1.0
9,201646.0,2.0


In [101]:
# c_big_2.sort_values(by='buy_numbers',ascending=False)
c_big_2 = pd.merge(will_buy,c_big_2,on='user_id',how='left').drop_duplicates()
# c_big_2 = c_big_2[['user_id','type_2','type_4','buy_numbers']].drop_duplicates()
c_big_2 = c_big_2[['user_id','type_2','buy_numbers']].drop_duplicates()
c_big_2.sort_values(by='buy_numbers',ascending=False)

,user_id,type_2,buy_numbers
419,214974.0,8.0,8.0
955,234715.0,4.0,7.0
1777,262503.0,1.0,7.0
291,210039.0,1.0,6.0
803,230308.0,9.0,5.0
856,232453.0,7.0,5.0
2352,284604.0,23.0,5.0
2663,299022.0,2.0,5.0
316,210911.0,2.0,5.0
2406,286592.0,3.0,5.0


In [103]:
c_big_2 = c
c_big_2.columns = ['user_id','buy_numbers']
with open('./cache/Action_2016-02-01_2016-04-16_with_time-type-user_id','rb') as f:
    user_action = pickle.load(f)
c_big_2 = pd.merge(c_big_2,user_action,on='user_id',how='left')
#4-14和4-15号的信息
c_big_2 = c_big_2[c_big_2.time>'2016-04-15 00:00:00']
c_big_2 = c_big_2[(c_big_2.type!=4)]
c_big_2 = c_big_2[c_big_2.type==2]
type_c = pd.get_dummies(c_big_2['type'],prefix='type')
will_buy = c_big_2['user_id']
will_buy = pd.DataFrame(will_buy)
will_buy = pd.concat([will_buy,type_c],axis=1)
will_buy = will_buy.groupby('user_id',as_index=False).sum()
# c_big_2.sort_values(by='buy_numbers',ascending=False)
c_big_2 = pd.merge(will_buy,c_big_2,on='user_id',how='left').drop_duplicates()
# c_big_2 = c_big_2[['user_id','type_2','type_4','buy_numbers']].drop_duplicates()
c_big_2 = c_big_2[['user_id','type_2','buy_numbers']].drop_duplicates()
c_big_2.sort_values(by='buy_numbers',ascending=False)

,user_id,type_2,buy_numbers
672,262503.0,1.0,7.0
113,210039.0,1.0,6.0
297,230308.0,9.0,5.0
116,210911.0,2.0,5.0
153,212687.0,1.0,5.0
179,219893.0,1.0,5.0
536,249903.0,8.0,5.0
44,205993.0,1.0,5.0
918,284604.0,20.0,5.0
596,256821.0,3.0,5.0


In [104]:
c_big_2.sort_values(by='type_2',ascending=False)

,user_id,type_2,buy_numbers
918,284604.0,20.0,5.0
984,293543.0,19.0,2.0
493,244664.0,16.0,2.0
276,229215.0,15.0,1.0
252,226699.0,13.0,1.0
211,223887.0,13.0,2.0
878,281292.0,12.0,1.0
142,212375.0,11.0,1.0
938,284628.0,11.0,4.0
327,232624.0,11.0,1.0


# 求所有用户的加购转化率，越低越好   5.9号代码

In [108]:
# with open('./cache/UCPair_action_cnt_2016-02-01_2016-04-16.pkl','rb') as f:
#     temp_1 = pickle.load(f)

temp_1.columns = ['user_id','cate','type','type_1','type_2','type_3','type_4','type_5','type_6']
temp_1 = temp_1[temp_1.cate==8]
temp_1['cart_buy_ratio'] = temp_1['type_2']/temp_1['type_4']
temp_1 = temp_1[['user_id','type_2','type_4','cart_buy_ratio']]
temp_1.sort_values(by='cart_buy_ratio',ascending=False)

D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,user_id,type_2,type_4,cart_buy_ratio
20,200016.0,4.0,0.0,inf
88920,267892.0,1.0,0.0,inf
89169,268070.0,3.0,0.0,inf
89142,268056.0,10.0,0.0,inf
89141,268051.0,1.0,0.0,inf
89137,268048.0,1.0,0.0,inf
89118,268034.0,1.0,0.0,inf
89065,267992.0,3.0,0.0,inf
89057,267981.0,1.0,0.0,inf
89024,267962.0,1.0,0.0,inf


In [114]:
temp_1 = temp_1[temp_1.type_4!=0.0]
temp_1.sort_values(by='cart_buy_ratio',ascending=True)

,user_id,type_2,type_4,cart_buy_ratio
37598,230032.0,0.0,1.0,0.0
64704,249165.0,0.0,1.0,0.0
63791,248515.0,0.0,1.0,0.0
63324,248152.0,0.0,1.0,0.0
28592,223751.0,0.0,1.0,0.0
63025,247899.0,0.0,1.0,0.0
10854,211302.0,0.0,1.0,0.0
103055,278917.0,0.0,1.0,0.0
103564,279340.0,0.0,1.0,0.0
29202,224173.0,0.0,1.0,0.0


In [149]:
temp_4 = pd.merge(temp_1,mm,on='user_id',how='left')
temp_4

,user_id,type_2,type_4,cart_buy_ratio,sku_id
0,200289.0,1.0,1.0,1.000000,NaN
1,200325.0,1.0,1.0,1.000000,NaN
2,200422.0,3.0,1.0,3.000000,NaN
3,200632.0,2.0,1.0,2.000000,NaN
4,200719.0,3.0,1.0,3.000000,NaN
5,200736.0,0.0,1.0,0.000000,NaN
6,200833.0,2.0,2.0,1.000000,NaN
7,200884.0,2.0,1.0,2.000000,NaN
8,200909.0,2.0,1.0,2.000000,NaN
9,200935.0,5.0,1.0,5.000000,NaN


In [151]:
temp_4[temp_4.sku_id>=0]

,user_id,type_2,type_4,cart_buy_ratio,sku_id
53,203295.0,11.0,1.0,11.0,202.0
162,216556.0,2.0,1.0,2.0,203.0
173,217581.0,5.0,1.0,5.0,204.0
176,217836.0,0.0,1.0,0.0,205.0
179,218150.0,1.0,1.0,1.0,206.0
225,222336.0,6.0,1.0,6.0,207.0
253,224173.0,0.0,1.0,0.0,208.0
281,226891.0,21.0,1.0,21.0,209.0
330,230787.0,2.0,1.0,2.0,210.0
344,231401.0,3.0,1.0,3.0,211.0


In [148]:
mm = m
mm.sort_values(by='user_id',ascending=True)

,user_id,sku_id
0,200288,0
1,200690,1
2,201568,2
0,203295,202
3,203534,3
4,204399,4
5,204642,5
6,206631,6
7,206995,7
8,207234,8


In [118]:
temp_1[temp_1.cart_buy_ratio>3]

,user_id,type_2,type_4,cart_buy_ratio
1386,200935.0,5.0,1.0,5.000000
1642,201090.0,8.0,1.0,8.000000
1711,201134.0,5.0,1.0,5.000000
1941,201292.0,4.0,1.0,4.000000
2319,201522.0,8.0,1.0,8.000000
2367,201560.0,6.0,1.0,6.000000
2472,201628.0,12.0,1.0,12.000000
2496,201647.0,16.0,3.0,5.333333
2885,201934.0,13.0,1.0,13.000000
2989,202014.0,4.0,1.0,4.000000


In [115]:
temp_1.describe()

,user_id,type_2,type_4,cart_buy_ratio
count,1091.000000,1091.000000,1091.000000,1091.000000
mean,251555.706691,3.189734,1.068744,3.046288
std,29787.909407,3.463271,0.331585,3.341255
min,200289.000000,0.000000,1.000000,0.000000
25%,226263.500000,1.000000,1.000000,1.000000
50%,250922.000000,2.000000,1.000000,2.000000
75%,276364.500000,4.000000,1.000000,4.000000
max,305214.000000,35.000000,6.000000,35.000000


In [142]:
temp

,user_id,type_2,buy_numbers
419,214974.0,8.0,8.0
955,234715.0,4.0,7.0
1777,262503.0,1.0,7.0
291,210039.0,1.0,6.0
803,230308.0,9.0,5.0
856,232453.0,7.0,5.0
2352,284604.0,23.0,5.0
2663,299022.0,2.0,5.0
316,210911.0,2.0,5.0
2406,286592.0,3.0,5.0


In [102]:
temp = c_big_2.sort_values(by='buy_numbers',ascending=False)

In [140]:
user_action_4

,user_id,type
16756,203295.0,2
60153,216556.0,2
66828,217581.0,2
67962,217836.0,2
68600,218150.0,2
85196,222336.0,2
97591,224173.0,2
106674,226891.0,2
125066,230787.0,2
128244,231401.0,2


In [141]:
# temp_3 = temp_1['user_id']
# temp_3 = pd.DataFrame(temp_3)
# with open('./cache/Action_2016-02-01_2016-04-16_with_time-type-user_id','rb') as f:
#     user_action_1 = pickle.load(f)
# user_action_2 = pd.merge(temp_3,user_action_1,on='user_id',how='left')
# user_action_3 = user_action_2[user_action_2.time>='2016-04-14 00:00:00']
# user_action_4=user_action_3[(user_action_3.type==2)|(user_action_3.type==4)][['user_id','type']].drop_duplicates()
user_action_5 = user_action_4.groupby('user_id',as_index=False).sum()
user_action_5 = user_action_5[user_action_5.type==2]
user_action_5

,user_id,type
0,203295.0,2
1,216556.0,2
2,217581.0,2
3,217836.0,2
4,218150.0,2
5,222336.0,2
6,224173.0,2
7,226891.0,2
8,230787.0,2
9,231401.0,2


In [62]:
c_big_2 = c_big_2[c_big_2.type_2>c_big_2.type_4]

In [143]:
# m = c_big_2['user_id']
m = user_action_5['user_id']
m = pd.DataFrame(m)
#匹配的时候，将购买转化率高的用户作为参考
# temp_2 = user_action_5
# m = pd.merge(m,temp_2,how='left',on='user_id')
# m
# m = pd.DataFrame(m)
l = pd.read_csv('副本a3.csv')
m = pd.concat([l,m],axis=0)
m = m['user_id']
m = pd.DataFrame(m)
m = m.drop_duplicates()
m['sku_id'] = [x for x in range(len(m))]
m = m.astype(int)
m

,user_id,sku_id
0,200288,0
1,200690,1
2,201568,2
3,203534,3
4,204399,4
5,204642,5
6,206631,6
7,206995,7
8,207234,8
9,207468,9


In [152]:
m.to_csv('pre_5_9_date_1.csv',index=False)    # 倘若还是没有几条要购买的用户  

# 结果是，这97条里面没一条在接下来5天购买了

In [1]:
# m.to_csv('pre_5_9_date_1.csv',index=False)    # 倘若还是没有几条要购买的用户  ,就转换思路

In [42]:
will_buy = will_buy.groupby('user_id',as_index=False).sum()

,user_id,2,4
0,202597.0,2.0,0.0
1,204424.0,1.0,0.0
2,206306.0,3.0,2.0
3,206731.0,2.0,0.0
4,208905.0,0.0,1.0
5,211730.0,2.0,1.0
6,212209.0,1.0,0.0
7,212293.0,15.0,2.0
8,212414.0,9.0,0.0
9,214690.0,1.0,0.0


In [24]:
c.type.describe()

count    652.000000
mean       2.498466
std        3.102201
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max       39.000000
Name: type, dtype: float64

In [19]:
#查看load_45_days行为数据
def load_days_about_buy():
    if os.path.exists('./cache/Action_load_45_60_days'):
        with open('./cache/Action_load_45_60_days','rb') as f:
            df =pickle.load(f)
    else:
        dump_path = './cache/Action_2016-02-01_2016-04-16_2'
        # if os.path.exists(dump_path):
        with open(dump_path,'rb') as f:
            df = pickle.load(f)
        user_info = pd.read_csv(USER_FILE,encoding='gbk')
        df= df[(df.load_days>45)&(df[df.load_days<=60])][['user_id','load_days']]
        #,'2016-04-11','2016-04-12','2016-04-13','2016-04-14','2016-04-15'
        df = pd.merge(df,user_info,how='left',on='user_id')
        with open('./cache/Action_2016-02-01_2016-04-16_with_time-type-user_id','rb') as f:
            user_action = pickle.load(f)
        df = pd.merge(df,user_action,how='left',on='user_id')
        df = df[df.type==4]
        with open('./cache/Action_load_45_60_days','wb') as f:
            pickle.dump(df,f)
    return df

In [18]:
load_days_about_buy()


D:\Anaconda3\lib\site-packages\pandas\indexes\base.py:2482: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)


MemoryError: 

In [22]:
load_60_days=df[df.load_days>60]

In [23]:
user_level = pd.read_csv(USER_FILE,encoding='gbk')
load_60_days = pd.merge(load_60_days,user_level,how='left',on='user_id')

load_60_days

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days,age,sex,user_lv_cd,user_reg_tm
0,201460.0,0,0,0,0,0,0,0,1,1,...,1,1,1,1,1,61,36-45岁,2.0,3,2016-02-08
1,201971.0,0,1,1,1,1,0,0,1,1,...,1,1,1,1,1,64,26-35岁,0.0,5,2008-11-07
2,203275.0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,73,26-35岁,0.0,4,2010-08-20
3,203940.0,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,63,26-35岁,0.0,4,2013-05-03
4,204488.0,1,1,0,1,1,1,1,0,0,...,0,0,1,1,0,61,26-35岁,1.0,5,2008-11-12
5,204977.0,1,1,1,1,0,1,1,0,1,...,1,0,0,1,1,61,26-35岁,0.0,5,2010-08-25
6,205515.0,1,0,0,1,1,0,1,1,1,...,0,1,0,1,1,67,16-25岁,0.0,4,2013-05-12
7,206084.0,1,1,1,1,1,1,1,1,1,...,1,1,0,1,0,66,26-35岁,0.0,5,2013-05-16
8,207005.0,1,0,1,1,0,0,0,1,0,...,1,1,1,1,1,64,26-35岁,0.0,5,2010-09-03
9,207809.0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,74,36-45岁,0.0,5,2008-11-22


In [24]:
load_60_days.sort_values(by=['user_lv_cd'],ascending=True)

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days,age,sex,user_lv_cd,user_reg_tm
0,201460.0,0,0,0,0,0,0,0,1,1,...,1,1,1,1,1,61,36-45岁,2.0,3,2016-02-08
120,298651.0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,64,16-25岁,2.0,3,2015-11-21
24,217713.0,1,0,1,0,0,0,0,0,0,...,1,1,1,1,1,64,26-35岁,0.0,3,2010-12-29
118,296107.0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,63,36-45岁,2.0,3,2015-11-07
55,237965.0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,71,56岁以上,0.0,3,2014-05-02
114,294771.0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,63,16-25岁,2.0,3,2015-10-30
113,293135.0,1,1,1,1,1,0,0,0,0,...,1,1,1,1,1,68,26-35岁,2.0,3,2015-10-19
107,290880.0,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,74,26-35岁,2.0,3,2012-12-26
40,229558.0,1,1,1,1,1,1,0,1,1,...,1,1,0,1,1,71,36-45岁,0.0,4,2014-03-07
44,232083.0,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,73,-1,2.0,4,2014-04-01


In [6]:
# with open('./cache/Action_2016-02-01_2016-04-16_with_time-type-user_id','rb') as f:
#     user_action = pickle.load(f)
# load_60_days_user = pd.merge(load_60_days,user_action,how='left',on='user_id')
# m = load_60_days_user
# load_60_days_user = load_60_days_user[load_60_days_user.type==4][['user_id','time','type','load_days','age','user_lv_cd','sex','user_reg_tm']]
# load_60_days_user

In [46]:
help(pd.DataFrame.reset_index)

Help on function reset_index in module pandas.core.frame:

reset_index(self, level=None, drop=False, inplace=False, col_level=0, col_fill='')
    For DataFrame with multi-level index, return new DataFrame with
    labeling information in the columns under the index names, defaulting
    to 'level_0', 'level_1', etc. if any are None. For a standard index,
    the index name will be used (if set), otherwise a default 'index' or
    'level_0' (if 'index' is already taken) will be used.
    
    Parameters
    ----------
    level : int, str, tuple, or list, default None
        Only remove the given levels from the index. Removes all levels by
        default
    drop : boolean, default False
        Do not try to insert index into dataframe columns. This resets
        the index to the default integer index.
    inplace : boolean, default False
        Modify the DataFrame in place (do not create a new object)
    col_level : int or str, default 0
        If the columns have multiple lev

In [16]:
with open('./cache/load_more_60_days_user_info','rb') as f:
    load_60_days_user = pickle.load(f)
load_60_days_user

,user_id,time,type,load_days,age,user_lv_cd,sex,user_reg_tm
6653,201971.0,2016-03-16 15:33:00,4,64,26-35岁,5,0.0,2008-11-07
10454,201971.0,2016-04-11 00:49:00,4,64,26-35岁,5,0.0,2008-11-07
10455,201971.0,2016-04-11 00:49:00,4,64,26-35岁,5,0.0,2008-11-07
11185,201971.0,2016-04-15 23:39:00,4,64,26-35岁,5,0.0,2008-11-07
29783,204488.0,2016-03-18 09:08:00,4,61,26-35岁,5,1.0,2008-11-12
38334,205515.0,2016-03-07 12:07:00,4,67,16-25岁,4,0.0,2013-05-12
42139,206084.0,2016-02-16 10:15:00,4,66,26-35岁,5,0.0,2013-05-16
45440,207005.0,2016-04-06 21:40:00,4,64,26-35岁,5,0.0,2010-09-03
80784,212090.0,2016-03-19 04:14:00,4,72,36-45岁,5,0.0,2013-07-18
88722,213882.0,2016-03-01 08:33:00,4,66,26-35岁,5,0.0,2010-11-01


In [25]:
load_60_days_user = load_60_days_user.sort_values(by='user_id',ascending=False)
load_60_days_user.reset_index(drop=True,inplace=False)

,user_id,time,type,load_days,age,user_lv_cd,sex,user_reg_tm
0,303392.0,2016-03-30 14:41:00,4,65,16-25岁,4,2.0,2016-01-08
1,301094.0,2016-02-29 11:13:00,4,67,36-45岁,4,2.0,2015-12-16
2,301094.0,2016-02-29 11:13:00,4,67,36-45岁,4,2.0,2015-12-16
3,301052.0,2016-02-13 13:51:00,4,62,36-45岁,4,2.0,2013-03-14
4,301052.0,2016-02-13 13:52:00,4,62,36-45岁,4,2.0,2013-03-14
5,301052.0,2016-02-13 17:04:00,4,62,36-45岁,4,2.0,2013-03-14
6,296588.0,2016-03-07 19:14:00,4,65,26-35岁,4,1.0,2015-11-10
7,296588.0,2016-03-01 16:43:00,4,65,26-35岁,4,1.0,2015-11-10
8,296107.0,2016-02-11 20:59:00,4,63,36-45岁,3,2.0,2015-11-07
9,295876.0,2016-03-31 09:01:00,4,62,26-35岁,5,2.0,2013-02-01


In [20]:
# load_60_days_user = load_60_days_user.sort_values(by='load_days',ascending=False)
# load_60_days_user.reset_index(drop=True,inplace=False)

,user_id,time,type,load_days,age,user_lv_cd,sex,user_reg_tm
0,243989.0,2016-03-10 16:14:00,4,74,36-45岁,5,0.0,2009-10-21
1,243989.0,2016-02-21 21:19:00,4,74,36-45岁,5,0.0,2009-10-21
2,243989.0,2016-04-13 19:11:00,4,74,36-45岁,5,0.0,2009-10-21
3,243989.0,2016-04-10 16:38:00,4,74,36-45岁,5,0.0,2009-10-21
4,243989.0,2016-02-12 18:26:00,4,74,36-45岁,5,0.0,2009-10-21
5,243989.0,2016-02-13 15:34:00,4,74,36-45岁,5,0.0,2009-10-21
6,243989.0,2016-02-14 13:38:00,4,74,36-45岁,5,0.0,2009-10-21
7,243989.0,2016-02-16 20:33:00,4,74,36-45岁,5,0.0,2009-10-21
8,243989.0,2016-02-19 11:02:00,4,74,36-45岁,5,0.0,2009-10-21
9,243989.0,2016-02-19 16:00:00,4,74,36-45岁,5,0.0,2009-10-21


In [26]:
load_60_days_user.iloc[29:59]

,user_id,time,type,load_days,age,user_lv_cd,sex,user_reg_tm
651829,294834.0,2016-03-24 21:27:00,4,65,-1,5,2.0,2015-10-31
648003,294834.0,2016-02-24 11:12:00,4,65,-1,5,2.0,2015-10-31
646636,294834.0,2016-02-19 09:28:00,4,65,-1,5,2.0,2015-10-31
650367,294834.0,2016-03-16 18:35:00,4,65,-1,5,2.0,2015-10-31
649907,294834.0,2016-03-08 16:15:00,4,65,-1,5,2.0,2015-10-31
646741,294834.0,2016-02-19 17:21:00,4,65,-1,5,2.0,2015-10-31
645952,294834.0,2016-02-02 22:10:00,4,65,-1,5,2.0,2015-10-31
647875,294834.0,2016-02-23 20:29:00,4,65,-1,5,2.0,2015-10-31
646282,294834.0,2016-02-16 16:14:00,4,65,-1,5,2.0,2015-10-31
646499,294834.0,2016-02-17 22:09:00,4,65,-1,5,2.0,2015-10-31


In [27]:
load_60_days_user.iloc[59:99]

,user_id,time,type,load_days,age,user_lv_cd,sex,user_reg_tm
646211,294834.0,2016-02-16 08:54:00,4,65,-1,5,2.0,2015-10-31
650433,294834.0,2016-03-20 08:19:00,4,65,-1,5,2.0,2015-10-31
646946,294834.0,2016-02-20 13:39:00,4,65,-1,5,2.0,2015-10-31
647298,294834.0,2016-02-22 12:05:00,4,65,-1,5,2.0,2015-10-31
641735,293135.0,2016-02-13 13:05:00,4,68,26-35岁,3,2.0,2015-10-19
641736,293135.0,2016-02-13 13:07:00,4,68,26-35岁,3,2.0,2015-10-19
640876,292813.0,2016-03-14 21:35:00,4,63,26-35岁,4,2.0,2013-01-10
639230,292813.0,2016-02-25 15:57:00,4,63,26-35岁,4,2.0,2013-01-10
639182,292813.0,2016-02-22 12:31:00,4,63,26-35岁,4,2.0,2013-01-10
633712,292301.0,2016-03-10 00:22:00,4,71,-1,5,0.0,2015-10-08


In [28]:
load_60_days_user.iloc[99:159]

,user_id,time,type,load_days,age,user_lv_cd,sex,user_reg_tm
587895,290054.0,2016-04-15 11:13:00,4,61,36-45岁,5,0.0,2012-12-19
586824,290054.0,2016-04-08 03:22:00,4,61,36-45岁,5,0.0,2012-12-19
587676,290054.0,2016-04-14 14:17:00,4,61,36-45岁,5,0.0,2012-12-19
587860,290054.0,2016-04-15 10:31:00,4,61,36-45岁,5,0.0,2012-12-19
586729,290054.0,2016-04-07 16:07:00,4,61,36-45岁,5,0.0,2012-12-19
585381,290054.0,2016-03-24 15:31:00,4,61,36-45岁,5,0.0,2012-12-19
586730,290054.0,2016-04-07 16:07:00,4,61,36-45岁,5,0.0,2012-12-19
583129,289645.0,2016-04-14 00:29:00,4,63,26-35岁,5,2.0,2012-12-15
576977,288734.0,2016-02-25 05:39:00,4,62,26-35岁,5,1.0,2012-12-10
577992,288734.0,2016-03-14 00:02:00,4,62,26-35岁,5,1.0,2012-12-10


In [50]:
find_number = load_60_days_user[['user_id','type']]
find_number

,user_id,type
710058,303392.0,4
693585,301094.0,4
693584,301094.0,4
682956,301052.0,4
682959,301052.0,4
683346,301052.0,4
676588,296588.0,4
675016,296588.0,4
662184,296107.0,4
658177,295876.0,4


In [48]:
type_dummies = pd.get_dummies(find_number['type'],prefix='type')
find_number1 = pd.concat([find_number,type_dummies],axis=1)[['user_id','type_4']]
find_number1 = find_number1.groupby('user_id',as_index=False).sum()

In [55]:
find_number1 = find_number1.sort_values('type_4',ascending=True)

In [59]:
ll = find_number1.iloc[70:]['user_id']
ll = pd.DataFrame(ll) 

In [43]:
load_60_days_user[load_60_days_user.age=='46-45岁']

,user_id,time,type,load_days,age,user_lv_cd,sex,user_reg_tm
299403,237793.0,2016-03-07 06:26:00,4,69,46-55岁,5,0.0,2011-10-16
308544,237793.0,2016-03-27 20:24:00,4,69,46-55岁,5,0.0,2011-10-16
309559,237793.0,2016-04-15 17:45:00,4,69,46-55岁,5,0.0,2011-10-16
381774,255206.0,2016-02-02 10:11:00,4,67,46-55岁,5,2.0,2014-09-22
381775,255206.0,2016-02-02 10:11:00,4,67,46-55岁,5,2.0,2014-09-22
381782,255206.0,2016-02-02 11:26:00,4,67,46-55岁,5,2.0,2014-09-22
381890,255206.0,2016-02-05 14:21:00,4,67,46-55岁,5,2.0,2014-09-22
382177,255206.0,2016-02-14 17:38:00,4,67,46-55岁,5,2.0,2014-09-22
382458,255206.0,2016-02-23 17:54:00,4,67,46-55岁,5,2.0,2014-09-22
382534,255206.0,2016-02-23 23:01:00,4,67,46-55岁,5,2.0,2014-09-22


In [34]:
with open('./cache/load_more_60_days_user_info','wb') as f:
    pickle.dump(load_60_days_user,f)

In [29]:
# delta_user = load_60_days_user['user_id']
# delta_user = pd.DataFrame(delta_user).drop_duplicates()
# load_60 = pd.merge(m,delta_user,on='user_id',how='left')
# load_60.fillna(value=0)
# load_60 = load_60[load_60.type!=0]['user_id']
# load_60 = pd.DataFrame(load_60)
load_60=load_60.drop_duplicates()

In [30]:
a = pd.read_csv('./a2.csv')['user_id']
a = pd.DataFrame(a)
a = pd.concat([a,load_60],axis=0)
a

,user_id
0,200094.0
1,200210.0
2,200288.0
3,200690.0
4,200958.0
5,201931.0
6,203534.0
7,204253.0
8,204359.0
9,204399.0


In [36]:
a['sku_id'] = [x for x in range(len(a))]
a = pd.DataFrame(a).astype(int)
a.to_csv('a4.csv',index=False)

In [37]:
c = pd.read_csv('a4.csv')
c

,user_id,sku_id
0,200094,0
1,200210,1
2,200288,2
3,200690,3
4,200958,4
5,201931,5
6,203534,6
7,204253,7
8,204359,8
9,204399,9


In [25]:
load_60_days_user['time']=pd.to_datetime(load_60_days_user['time'])
load_60_days_user

6653     2016-03-16 15:33:00
10454    2016-04-11 00:49:00
10455    2016-04-11 00:49:00
11185    2016-04-15 23:39:00
29783    2016-03-18 09:08:00
38334    2016-03-07 12:07:00
42139    2016-02-16 10:15:00
45440    2016-04-06 21:40:00
80784    2016-03-19 04:14:00
88722    2016-03-01 08:33:00
88723    2016-03-01 08:33:00
90557    2016-04-12 21:25:00
91596    2016-02-18 17:31:00
94834    2016-03-19 14:25:00
101363   2016-03-28 18:12:00
109553   2016-03-21 10:21:00
121366   2016-03-04 23:53:00
121658   2016-03-06 20:24:00
123625   2016-03-29 12:29:00
123626   2016-03-29 12:29:00
123627   2016-03-29 12:29:00
123773   2016-03-31 15:10:00
123853   2016-04-01 18:02:00
124548   2016-04-14 00:11:00
125381   2016-02-05 16:24:00
127108   2016-03-11 17:59:00
131647   2016-03-07 21:56:00
134739   2016-03-30 16:34:00
136372   2016-04-11 00:17:00
151183   2016-04-08 23:00:00
                 ...        
650021   2016-03-10 11:29:00
650135   2016-03-10 16:27:00
650214   2016-03-13 11:30:00
650367   2016-

In [24]:
df[df.load_days>10]          #此方法没用，这个特征并不理想

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days
0,200001.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,1,22
1,200002.0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,19
2,200003.0,0,1,1,0,0,0,0,0,0,...,1,0,0,1,0,0,1,1,0,13
7,200008.0,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,12
11,200012.0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,0,1,14
12,200013.0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,15
13,200014.0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,1,0,21
14,200015.0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,1,1,0,1,21
15,200016.0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,12
21,200022.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,12


In [6]:
dump_path = './cache/Action_2016-02-01_2016-04-16_0'
if os.path.exists(dump_path):
    with open(dump_path,'rb') as f:
        df = pickle.load(f)
df

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,last_7_sum,all_days_sum
0,200001.0,0,0,0,0,0,0,0,0,0,...,19,6,0,1,0,2,0,1,9,671.0
1,200002.0,16,7,46,0,0,0,0,0,0,...,0,0,0,0,0,92,41,0,133,856.0
2,200003.0,0,31,20,0,0,0,0,0,0,...,0,0,11,0,0,23,25,0,59,706.0
3,200004.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,43,0,34,0,77,190.0
4,200005.0,0,0,0,0,0,0,0,0,0,...,0,36,0,0,0,0,0,0,36,311.0
5,200006.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7,0,369.0
6,200007.0,0,0,0,0,0,22,0,4,0,...,0,0,26,0,0,0,0,0,26,818.0
7,200008.0,28,0,0,0,0,29,0,21,0,...,0,0,0,0,0,0,0,0,0,311.0
8,200009.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,2,3,4,15.0
9,200010.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,90.0


In [6]:
#查看用户等级高的用户的行为数据
def user_level_high():
    dump_path = './cache/Action_high_user_level'
    if os.path.exists(dump_path):
        with open(dump_path,'rb') as f:
            df = pickle.load(f)
    else:
        df = pd.read_csv(USER_FILE,encoding='gbk')
        df = df[df.user_lv_cd==5]
        with open('./cache/Action_2016-02-01_2016-04-16_0','rb') as f:
            browse_info = pickle.load(f)
        df = pd.merge(df,browse_info,how='left',on='user_id')
        with open(dump_path,'wb') as f:
            pickle.dump(df,f)
    return df

In [8]:
user_level_high()

,user_id,age,sex,user_lv_cd,user_reg_tm,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,...,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,last_7_sum,all_days_sum
0,200001,56岁以上,2.0,5,2016-01-26,0.0,0.0,0.0,0.0,0.0,...,19.0,6.0,0.0,1.0,0.0,2.0,0.0,1.0,9.0,671.0
1,200011,-1,0.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.0
2,200022,26-35岁,0.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,12.0,153.0
3,200027,26-35岁,1.0,5,2013-04-10,0.0,0.0,0.0,24.0,3.0,...,0.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,666.0
4,200033,36-45岁,2.0,5,2016-02-02,0.0,18.0,0.0,0.0,7.0,...,96.0,99.0,0.0,73.0,1.0,7.0,136.0,8.0,316.0,1009.0
5,200037,26-35岁,2.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,17.0,63.0,0.0,0.0,4.0,80.0,329.0
6,200038,26-35岁,0.0,5,2013-04-10,84.0,24.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,45.0,25.0,45.0,3062.0
7,200045,26-35岁,2.0,5,2013-04-10,0.0,0.0,90.0,12.0,73.0,...,0.0,20.0,0.0,9.0,22.0,7.0,26.0,0.0,84.0,833.0
8,200054,16-25岁,2.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,31.0,0.0,250.0,10.0,80.0,23.0,230.0,336.0,593.0,1902.0
9,200059,26-35岁,2.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,109.0,14.0,7.0,0.0,17.0,6.0,0.0,33.0,44.0,484.0


In [7]:
user_dataset = user_level_high()
user_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36088 entries, 0 to 36087
Data columns (total 82 columns):
user_id         36088 non-null int64
age             36088 non-null object
sex             36088 non-null float64
user_lv_cd      36088 non-null int64
user_reg_tm     36088 non-null object
2016-02-01      36048 non-null float64
2016-02-02      36048 non-null float64
2016-02-03      36048 non-null float64
2016-02-04      36048 non-null float64
2016-02-05      36048 non-null float64
2016-02-06      36048 non-null float64
2016-02-07      36048 non-null float64
2016-02-08      36048 non-null float64
2016-02-09      36048 non-null float64
2016-02-10      36048 non-null float64
2016-02-11      36048 non-null float64
2016-02-12      36048 non-null float64
2016-02-13      36048 non-null float64
2016-02-14      36048 non-null float64
2016-02-15      36048 non-null float64
2016-02-16      36048 non-null float64
2016-02-17      36048 non-null float64
2016-02-18      36048 non-null float64


In [8]:
load_user = user_dataset[(user_dataset.last_7_sum!=0)]
load_user['last_7_days_ratio'] = load_user['last_7_sum']/load_user['all_days_sum']
load_user

D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,user_id,age,sex,user_lv_cd,user_reg_tm,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,...,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,last_7_sum,all_days_sum,last_7_days_ratio
0,200001,56岁以上,2.0,5,2016-01-26,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,1.0,0.0,2.0,0.0,1.0,9.0,671.0,0.013413
2,200022,26-35岁,0.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,0.0,0.0,0.0,0.0,12.0,153.0,0.078431
3,200027,26-35岁,1.0,5,2013-04-10,0.0,0.0,0.0,24.0,3.0,...,52.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,666.0,0.078078
4,200033,36-45岁,2.0,5,2016-02-02,0.0,18.0,0.0,0.0,7.0,...,99.0,0.0,73.0,1.0,7.0,136.0,8.0,316.0,1009.0,0.313181
5,200037,26-35岁,2.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,17.0,63.0,0.0,0.0,4.0,80.0,329.0,0.243161
6,200038,26-35岁,0.0,5,2013-04-10,84.0,24.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,45.0,25.0,45.0,3062.0,0.014696
7,200045,26-35岁,2.0,5,2013-04-10,0.0,0.0,90.0,12.0,73.0,...,20.0,0.0,9.0,22.0,7.0,26.0,0.0,84.0,833.0,0.100840
8,200054,16-25岁,2.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,0.0,250.0,10.0,80.0,23.0,230.0,336.0,593.0,1902.0,0.311777
9,200059,26-35岁,2.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,14.0,7.0,0.0,17.0,6.0,0.0,33.0,44.0,484.0,0.090909
10,200068,56岁以上,0.0,5,2016-01-27,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,6.0,21.0,13.0,43.0,41.0,351.0,0.116809


In [9]:
load_user.sort_values(by=['all_days_sum'],ascending=False).dropna(axis=0)

,user_id,age,sex,user_lv_cd,user_reg_tm,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,...,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,last_7_sum,all_days_sum,last_7_days_ratio
27582,271592,36-45岁,0.0,5,2015-03-20,44.0,403.0,31.0,145.0,256.0,...,758.0,451.0,151.0,60.0,350.0,897.0,257.0,2667.0,22861.0,0.116662
15266,237793,46-55岁,0.0,5,2011-10-16,12.0,0.0,83.0,21.0,755.0,...,11.0,57.0,26.0,0.0,0.0,41.0,47.0,135.0,22191.0,0.006084
34351,296877,36-45岁,0.0,5,2015-11-11,0.0,0.0,0.0,0.0,0.0,...,308.0,129.0,645.0,548.0,350.0,785.0,1151.0,2765.0,21813.0,0.126759
15692,238543,26-35岁,1.0,5,2011-10-28,348.0,0.0,420.0,895.0,368.0,...,113.0,3.0,454.0,287.0,191.0,468.0,334.0,1516.0,20472.0,0.074052
3244,212090,36-45岁,0.0,5,2013-07-18,148.0,86.0,6.0,32.0,10.0,...,761.0,292.0,321.0,128.0,151.0,1216.0,86.0,2869.0,19284.0,0.148776
26841,269325,26-35岁,0.0,5,2012-07-20,617.0,78.0,136.0,68.0,101.0,...,469.0,735.0,133.0,75.0,192.0,948.0,510.0,2552.0,16747.0,0.152386
29981,279524,26-35岁,0.0,5,2010-04-09,0.0,0.0,0.0,41.0,0.0,...,1157.0,131.0,4.0,172.0,85.0,39.0,79.0,1588.0,16438.0,0.096605
1143,207081,26-35岁,0.0,5,2013-05-25,1103.0,904.0,222.0,80.0,8.0,...,138.0,0.0,0.0,0.0,0.0,0.0,6.0,138.0,16286.0,0.008474
15510,238222,26-35岁,0.0,5,2011-10-23,0.0,5.0,0.0,0.0,0.0,...,227.0,2153.0,3166.0,467.0,0.0,83.0,0.0,6096.0,16224.0,0.375740
25517,265648,26-35岁,2.0,5,2012-06-13,0.0,0.0,0.0,0.0,0.0,...,276.0,150.0,404.0,191.0,294.0,411.0,14.0,1726.0,15880.0,0.108690


In [13]:
user_id_5 = load_user[['user_id','last_7_days_ratio']].dropna(axis=0)
with open('./cache/Action_2016-02-01_2016-04-16_with_cate-time-type-user_id','rb') as f:
    all_data =pickle.load(f)
all_data = all_data[all_data.cate ==8][['user_id','type','time']]
user_id_5 = pd.merge(user_id_5,all_data,how='left',on='user_id')[['user_id','last_7_days_ratio','type','time']]
user_id_5

,user_id,last_7_days_ratio,type,time
0,200001,0.013413,NaN,NaN
1,200022,0.078431,6.0,2016-02-22 18:55:16
2,200022,0.078431,1.0,2016-02-22 18:55:16
3,200022,0.078431,6.0,2016-02-22 18:55:16
4,200022,0.078431,6.0,2016-02-22 18:55:26
5,200022,0.078431,6.0,2016-02-22 18:55:50
6,200022,0.078431,6.0,2016-02-22 18:56:19
7,200022,0.078431,1.0,2016-03-02 22:17:11
8,200022,0.078431,6.0,2016-03-02 22:17:12
9,200022,0.078431,1.0,2016-03-02 22:17:19


In [14]:
user_id_5.dropna(axis=0)

,user_id,last_7_days_ratio,type,time
1,200022,0.078431,6.0,2016-02-22 18:55:16
2,200022,0.078431,1.0,2016-02-22 18:55:16
3,200022,0.078431,6.0,2016-02-22 18:55:16
4,200022,0.078431,6.0,2016-02-22 18:55:26
5,200022,0.078431,6.0,2016-02-22 18:55:50
6,200022,0.078431,6.0,2016-02-22 18:56:19
7,200022,0.078431,1.0,2016-03-02 22:17:11
8,200022,0.078431,6.0,2016-03-02 22:17:12
9,200022,0.078431,1.0,2016-03-02 22:17:19
10,200022,0.078431,6.0,2016-03-02 22:17:20


In [15]:
user_id_5[user_id_5.type==4.0]

,user_id,last_7_days_ratio,type,time
1040,200074,0.161972,4.0,2016-03-04 14:49:00
3177,200299,0.004768,4.0,2016-03-01 15:31:00
6418,200683,0.074627,4.0,2016-03-03 18:57:00
6943,200833,0.036810,4.0,2016-02-01 20:05:00
6944,200833,0.036810,4.0,2016-02-01 20:05:00
8603,201048,0.023790,4.0,2016-03-07 12:56:00
11584,201444,0.044510,4.0,2016-02-21 20:22:00
11716,201457,0.057665,4.0,2016-03-09 20:32:00
16891,202633,0.006369,4.0,2016-02-06 18:46:00
20247,203144,0.335861,4.0,2016-02-22 14:57:00


In [63]:
c = pd.read_csv('pre_5_07_2.csv')
# a = pd.read_csv('a2.csv')
a = pd.read_csv('./新建文件夹/a1.csv')
m =pd.merge(c,a,how='left',on='user_id')
l = m.loc[m.sku_id_y>0]


In [64]:
l = l[['user_id','sku_id_x']]
l.columns = ['user_id','sku_id']
l.to_csv('a3副本.csv',index=False)

In [66]:
l = pd.concat([l,ll],axis=0)['user_id']
l =pd.DataFrame(l).astype(int)
l = l.drop_duplicates()
l['sku_id'] = [x for x in range(len(l))]
l.to_csv('副本a3.csv',index=False)

In [67]:
len(l)

202

In [11]:
with open('./cache/Action_2016-02-01_2016-04-16_with_cate-type-user_id','rb') as f:
    m = pickle.load(f)
len(m['user_id'].drop_duplicates())

59790

In [13]:
#查看用户等级高的用户的行为数据
def user_level_high():
    dump_path = './cache/Action_high_user_level_1'
    if os.path.exists(dump_path):
        with open(dump_path,'rb') as f:
            df = pickle.load(f)
    else:
        df = pd.read_csv(USER_FILE,encoding='gbk')
        df = df[df.user_lv_cd==5]
        with open('./cache/Action_2016-02-01_2016-04-16_2','rb') as f:
            browse_info = pickle.load(f)
        df = pd.merge(df,browse_info,how='left',on='user_id')
        with open(dump_path,'wb') as f:
            pickle.dump(df,f)
    return df

In [14]:
load_user_1 = user_level_high()
load_user_1 = load_user_1[load_user_1.load_days>0]
load_user_1

,user_id,age,sex,user_lv_cd,user_reg_tm,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,...,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days
0,200001,56岁以上,2.0,5,2016-01-26,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,22.0
1,200011,-1,0.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,200022,26-35岁,0.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,12.0
3,200027,26-35岁,1.0,5,2013-04-10,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0
4,200033,36-45岁,2.0,5,2016-02-02,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,16.0
5,200037,26-35岁,2.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,11.0
6,200038,26-35岁,0.0,5,2013-04-10,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,35.0
7,200045,26-35岁,2.0,5,2013-04-10,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,37.0
8,200054,16-25岁,2.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,15.0
9,200059,26-35岁,2.0,5,2013-04-10,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,22.0


In [29]:
b = last_day_load_and_outnumber_two_days()
b.to_csv('a.csv',index=False)

In [36]:
not_buy_user()

,user_id
0,200094
1,200210
2,200288
3,200690
5,200958
7,201568
9,201931
10,203534
11,204014
12,204253


In [2]:
# #数据扩充
# def add_data():
#     dump_path = './cache/Action_2016-02-01_2016-04-16_2'
#     with open(dump_path,'rb') as f:
#         df = pickle.load(f)
#     #最后5天登陆过，并且两个月登陆次数很高或者最后5天登陆过，并且加入购物车的用户
#     df1 = df[['2016-04-11','2016-04-12','2016-04-13','2016-04-14','2016-04-15']]
#     df['last_five_day_load'] = df1.sum(axis=0)
#     last_five_days = df[(df.last_five_day_load>=2)&(df.load_days>10)]['user_id']
#     last_five_days = pd.DataFrame(last_five_days)
#     #最后5天登陆了并购买了的用户
#     path = './cache/Action_2016-02-01_2016-04-16_with_sku_id-time-type-user_id'
#     with open(path,'rb') as f:
#         df2 = pickle.load(f)
#     last_five_date = df2[df2.time>='2016-04-11']
#     last_five_date = last_five_date[last_five_date.type==4][['user_id','sku_id']]
#     trust_buy = pd.merge(last_five_days,last_five_date,on='user_id',how='left')
#     trust_buy = trust_buy.fillna(value=0)
#     user_id = trust_buy[trust_buy.sku_id==0]['user_id']
#     user_id = pd.DataFrame(user_id) 
#     with open('./cache/after_some_day_buy','wb') as f:
#         pickle(user_id,f)
#     return user_id

#符合要求的太多

In [ ]:
add_data()

In [6]:
m = pd.read_csv('./新建文件夹/a1.csv')
m

,user_id,sku_id
0,200094,0
1,200210,1
2,200288,2
3,200690,3
4,200958,4
5,201568,5
6,201931,6
7,203534,7
8,204014,8
9,204253,9


In [8]:
pp = pd.read_csv('./datafile/JData_Product.csv')
pp

,sku_id,a1,a2,a3,cate,brand
0,10,3,1,1,8,489
1,100002,3,2,2,8,489
2,100003,1,-1,-1,8,30
3,100006,1,2,1,8,545
4,10001,-1,1,2,8,244
5,100016,3,-1,2,8,214
6,100029,3,2,2,8,214
7,10003,3,1,2,8,214
8,100045,2,2,2,8,124
9,100057,3,1,2,8,306


In [65]:
with open('./cache/Action_2016-02-01_2016-04-16_with_sku_id-time-type-user_id','rb') as f:
    cc = pickle.load(f)
cc = cc[['user_id','type']]
cc = pd.DataFrame(cc).astype(int)
cc = pd.merge(m,cc,on='user_id',how='left')[['user_id','type']].drop_duplicates()
cc = cc.dropna(axis=0)
cc = cc[cc.type==3]
cc.reset_index(inplace=True)

In [60]:
cc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 716 entries, 0 to 715
Data columns (total 3 columns):
user_id    716 non-null int64
sku_id     716 non-null int64
type       354 non-null float64
dtypes: float64(1), int64(2)
memory usage: 22.4 KB


In [42]:
mm.last_two_days.describe()

count    381.000000
mean      46.713911
std       88.242367
min        1.000000
25%        7.000000
50%       20.000000
75%       51.000000
max      995.000000
Name: last_two_days, dtype: float64

In [21]:
nn = mm[(mm.last_two_days<446)&(mm.last_two_days>=4)]
nn

,user_id,2016-04-14,2016-04-15,last_two_days
0,200094,0,6,6
1,200210,0,25,25
2,200288,0,140,140
3,200690,0,5,5
4,200958,0,28,28
6,201931,0,20,20
7,203534,0,127,127
9,204253,15,54,69
10,204359,0,4,4
11,204399,0,6,6


In [22]:
nn = nn['user_id']
nn = pd.DataFrame(nn)
nn['sku_id'] = [x for x in range(len(nn))]
nn.to_csv('a2.csv',index=False)

In [19]:
len(mm[mm.last_two_days<=4])

37

In [10]:
mm.sort_values('last_two_days',ascending=False)

,user_id,2016-04-14,2016-04-15,last_two_days
38,207458,0,995,995
46,207511,0,777,777
329,294735,0,446,446
332,295066,71,323,394
93,220271,0,393,393
266,276780,0,352,352
380,305287,247,102,349
58,208003,0,335,335
303,286634,0,293,293
35,207450,157,136,293


In [9]:
mm=pd.merge(m,df,on='user_id',how='left')[['user_id','2016-04-14','2016-04-15']]
mm['last_two_days'] = mm.sum(axis=1) -mm.user_id
mm

,user_id,2016-04-14,2016-04-15,last_two_days
0,200094,0,6,6
1,200210,0,25,25
2,200288,0,140,140
3,200690,0,5,5
4,200958,0,28,28
5,201568,0,3,3
6,201931,0,20,20
7,203534,0,127,127
8,204014,0,2,2
9,204253,15,54,69


In [51]:
last_two_days = df
#last_two_days.columns = [x for x in range(77)]
#last_two_days = last_two_days.rename({0:'user_id',76:'load_days'},inplace=True)
last_two_days = last_two_days[(last_two_days.load_days<=2)&(last_two_days.loc[:,'2016-04-15']==1)]
last_two_days

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days
93,200094.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
209,200210.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
287,200288.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
689,200690.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
807,200808.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
957,200958.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
1175,201176.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1567,201568.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1671,201672.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
1929,201931.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2


In [85]:
buy_user_id = last_two_days['user_id']
buy_user_id = pd.DataFrame(buy_user_id)
buy_user_id

,user_id
93,200094.0
209,200210.0
287,200288.0
689,200690.0
807,200808.0
957,200958.0
1175,201176.0
1567,201568.0
1671,201672.0
1929,201931.0


In [87]:
buy_user_id['sku_id'] = [x for x in range(len(buy_user_id))]
buy_user_id.user_id = buy_user_id['user_id'].astype(int)
buy_user_id.to_csv('aa.csv',index=False)

In [66]:
path = './cache/Action_2016-02-01_2016-04-16_with_sku_id-time-user_id'
with open(path,'rb') as f:
    buy_sku_id = pickle.load(f)
buy_sku_id
# buy_user_sku_id = pd.merge(buy_user_id,buy_sku_id,on='user_id',how='left').drop_duplicates()
# buy_user_sku_id

,sku_id,time,user_id
0,107774,2016-02-01 00:00:00,272629.0
1,107774,2016-02-01 00:00:00,272629.0
2,107774,2016-02-01 00:00:00,272629.0
3,107774,2016-02-01 00:00:00,272629.0
4,107774,2016-02-01 00:00:00,272629.0
5,107774,2016-02-01 00:00:00,272629.0
6,107774,2016-02-01 00:00:00,272629.0
7,87598,2016-02-01 00:00:01,284046.0
8,153817,2016-02-01 00:00:01,208720.0
9,87598,2016-02-01 00:00:01,284046.0


In [68]:
buy_sku_id = buy_sku_id.drop_duplicates()
buy_user_sku_id = pd.merge(buy_user_id,buy_sku_id,on='user_id',how='left').drop_duplicates()
buy_user_sku_id

,user_id,sku_id,time
0,200094.0,47193,2016-04-15 15:39:52
1,200210.0,211,2016-03-16 01:20:17
2,200210.0,211,2016-03-16 01:20:18
3,200210.0,55497,2016-03-16 01:20:49
4,200210.0,211,2016-03-16 01:20:49
5,200210.0,211,2016-03-16 01:20:53
6,200210.0,89098,2016-03-16 01:23:12
7,200210.0,89098,2016-03-16 01:23:13
8,200210.0,89098,2016-03-16 01:23:47
9,200210.0,107012,2016-03-16 01:29:03


In [71]:
buy_user_sku_id['time'] = buy_user_sku_id.time.apply(lambda t :t[:10])
buy_user_sku_id = buy_user_sku_id[buy_user_sku_id.time>='2016-04-14'][['user_id','sku_id']]
buy_user_sku_id

D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,user_id,sku_id
0,200094.0,47193
16,200210.0,41390
17,200210.0,25218
18,200210.0,79937
19,200210.0,6851
20,200210.0,6851
21,200210.0,41390
22,200288.0,166707
23,200288.0,166707
24,200288.0,166707


In [82]:
# buy_user_sku_id = buy_user_sku_id.groupby(['user_id','sku_id']).sku_id.count()
# buy_user_sku_id = pd.DataFrame(buy_user_sku_id)
buy_user_sku_id

sku_id
user_id  sku_id        
200094.0 47193        1
200210.0 6851         2
         25218        1
         41390        2
         79937        1
200288.0 18412        3
         39387        1
         116246       1
         130900       1
         131300       1
         149854       3
         166707      16
200690.0 149641       1
200808.0 154636       6
200958.0 20308        8
         44854        3
         132782       4
         142140       7
         154636       1
201176.0 31662       12
         44854       15
         65520        3
201568.0 12564        3
201672.0 79520       11
         81708        1
         147796       1
201931.0 6033         9
         113997       4
203534.0 20308        9
         32465        7
...                 ...
305131.0 84544        2
305197.0 52343        2
305287.0 4285         2
         4723         4
         12540        6
         14163        1
         15259        1
         17242        1
         24371        6
         37957        4
         37995        1
         47193        3
         57161        2
         63434        1
         66169        1
         75824        2
         77221        1
         78335        2
         81462        1
         83148        2
         84409        3
         103132       1
         103652       2
         104468       1
         128567       1
         142543       2
         145990       1
         149854       5
         159528       2
         160750       2

[2395 rows x 1 columns]

In [ ]:
m = buy_user_sku_id

In [3]:
# #
# def get_fifty_days_data(start_date ='2016-02-01 00:00:00',end_date='2016-02-16 00:00:00'):
#     dump_path = './cache/Action_fifty_days_{0}_{1}'.format(start_date[:10],end_date[:10])
#     if os.path.exists(dump_path):
#         with open(dump_path,'rb') as f:
#             df = pickle.load(f)
#     else:
#         with open('./cache/Action_2016-02-01_2016-04-16_with_time-type-user_id','rb') as m:
#             df = pickle.load(m)
#         df = df[(df.time<end_date)&(df.time>start_date)]
#         df['time'] = df.time.apply(lambda t: t[:10])
#         type_dummies = pd.get_dummies(df['type'],prefix='Action_user')
#         user_data = pd.concat(df,type_dummies,axis=1)
        
#         user_data = user_data.groupby(['user_id','time'],as_index=False).sum
#         user_data = user_data[user_data.Action_user_4>0][['user_id','time']]
#         user_data = pd.merge(user_data,df,how='left',on=['user_id','time'])
#         dd = user_data.dropna(axis=0)
        

In [14]:

# a =np.random.randn(10,4)

# b = pd.DataFrame(a,columns=['a','b','c','d'])
# b.iloc[3:7,2:4] =np.nan
# b.dropna(axis=0)

,a,b,c,d
0,0.367581,-0.718925,0.419468,-0.118477
1,0.383625,-0.022122,0.822760,0.552877
2,0.509588,1.310190,0.881322,-1.206136
7,-0.450910,1.664112,1.207873,-0.311591
8,0.053854,-1.014994,0.694994,0.958569
9,1.321522,0.304714,2.439444,-0.145038
